In [1]:
import MongoDB as mdb
import ETC_BaseModul as bm
import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
from utils import unroll_ts
from model import hyperparameters
from orion.primitives.tadgan import TadGAN
import crawler_tosRS
from datetime import datetime, timedelta

Using TensorFlow backend.


In [2]:
def time_segments_aggregate(X, interval, time_column, method=['mean']):
    """Aggregate values over given time span.
    Args:
        X (ndarray or pandas.DataFrame):
            N-dimensional sequence of values.
        interval (int):
            Integer denoting time span to compute aggregation of.
        time_column (int):
            Column of X that contains time values.
        method (str or list):
            Optional. String describing aggregation method or list of strings describing multiple
            aggregation methods. If not given, `mean` is used.
    Returns:
        ndarray, ndarray:
            * Sequence of aggregated values, one column for each aggregation method.
            * Sequence of index values (first index of each aggregated segment).
    """
    if isinstance(X, np.ndarray):
        X = pd.DataFrame(X)

    X = X.sort_values(time_column).set_index(time_column)

    if isinstance(method, str):
        method = [method]

    start_ts = X.index.values[0]
    max_ts = X.index.values[-1]

    values = list()
    index = list()
    while start_ts <= max_ts:
        end_ts = start_ts + interval
        subset = X.loc[start_ts:end_ts - 1]
        aggregated = [
            getattr(subset, agg)(skipna=True).values
            for agg in method
        ]
        values.append(np.concatenate(aggregated))
        index.append(start_ts)
        start_ts = end_ts

    return np.asarray(values), np.asarray(index)

In [3]:
def rolling_window_sequences(X, index, window_size, target_size, step_size, target_column,
                             drop=None, drop_windows=False):
    """Create rolling window sequences out of time series data.
    The function creates an array of input sequences and an array of target sequences by rolling
    over the input sequence with a specified window.
    Optionally, certain values can be dropped from the sequences.
    Args:
        X (ndarray):
            N-dimensional sequence to iterate over.
        index (ndarray):
            Array containing the index values of X.
        window_size (int):
            Length of the input sequences.
        target_size (int):
            Length of the target sequences.
        step_size (int):
            Indicating the number of steps to move the window forward each round.
        target_column (int):
            Indicating which column of X is the target.
        drop (ndarray or None or str or float or bool):
            Optional. Array of boolean values indicating which values of X are invalid, or value
            indicating which value should be dropped. If not given, `None` is used.
        drop_windows (bool):
            Optional. Indicates whether the dropping functionality should be enabled. If not
            given, `False` is used.
    Returns:
        ndarray, ndarray, ndarray, ndarray:
            * input sequences.
            * target sequences.
            * first index value of each input sequence.
            * first index value of each target sequence.
    """
    out_X = list()
    out_y = list()
    X_index = list()
    y_index = list()
    target = X[:, target_column]

    if drop_windows:
        if hasattr(drop, '__len__') and (not isinstance(drop, str)):
            if len(drop) != len(X):
                raise Exception('Arrays `drop` and `X` must be of the same length.')
        else:
            if isinstance(drop, float) and np.isnan(drop):
                drop = np.isnan(X)
            else:
                drop = X == drop

    start = 0
    max_start = len(X) - window_size - target_size + 1
    while start < max_start:
        end = start + window_size

        if drop_windows:
            drop_window = drop[start:end + target_size]
            to_drop = np.where(drop_window)[0]
            if to_drop.size:
                start += to_drop[-1] + 1
                continue

        out_X.append(X[start:end])
        out_y.append(target[end:end + target_size])
        X_index.append(index[start])
        y_index.append(index[end])
        start = start + step_size

    return np.asarray(out_X), np.asarray(out_y), np.asarray(X_index), np.asarray(y_index)

In [4]:
RE_MAX_TIME = 3
before_N = 're_2'

target_N = 're_3'
max_count = 10

model_path = 'D:\\DeepLeaning\\TadGAN_model\\5-2. ver_15per(new)'
model_name = '5_2_15per(new)'

save_path = 'D:\\DeepLeaning\\TadGAN\\Shin\\Dataset\\realtime_test'
target_date = '2017-05-17'
save_name = 'result'

# 시뮬레이션..이니깐
year = target_date.split("-")[0]
month = target_date.split("-")[1]
day = target_date.split("-")[2]

In [5]:
# 실시간 최저점 찾기.
# 찐 시작
sunRS_df = crawler_tosRS.tosRS(year, month, day)

AM_term = [12.67192225, 11.48028811, 12.10103828, 9.321626622, 10.58524961, 10.28859822, 14.85200124, 9.099921203,
           9.895454429, 8.886125451, 11.34483191, 11.67173358]
PM_term = [13.97183439, 12.1092905, 10.90653908, 8.674959456, 10.48454171, 10.1388775, 12.01368493, 7.212434035,
           10.37609078, 7.496078472, 10.80341137, 13.25091053]

date = str(sunRS_df['Date'][0])
sunrise = str(sunRS_df['Sunup_Time'][0]).split(".")[0]
sunset = str(sunRS_df['Sundown_Time'][0]).split(".")[0]

# ori_date = year + "/" + month + "/" + day
start_time = ori_date + " " + sunrise + ":00"
end_time = ori_date + " " + sunset + ":00"

print(start_time)
start_time_obj = datetime.strptime(start_time, '%Y/%m/%d %H:%M:%S') - timedelta(hours=9)
end_time_obj = datetime.strptime(end_time, '%Y/%m/%d %H:%M:%S') - timedelta(hours=9)

day_len = (end_time_obj - start_time_obj).seconds / 60
am_ratio = AM_term[int(month) - 1] / 100
pm_ratio = PM_term[int(month) - 1] / 100
am_cut_line = start_time_obj + timedelta(minutes=int(day_len * am_ratio))
pm_cut_line = end_time_obj - timedelta(minutes=int(day_len * pm_ratio))
start_time = start_time_obj.strftime('%Y/%m/%d %H:%M:%S')
end_time = end_time_obj.strftime('%Y/%m/%d %H:%M:%S')
# 최저 시작점 찾기
min1 = 1000000
count = 0
start_min_index = 0

WebDriverException: Message: 'predict_cct_use_tadgan_generatorchromedriver' executable needs to be in PATH. Please see https://chromedriver.chromium.org/home


In [ ]:
while am_cut_line > start_time_obj:
    step_times = mdb.make_ktc_to_utc_4step1(start_time)
    step_data = mdb.load_last1_cct1(step_times)

    step_df = mdb.mongodb_to_df(step_data, 'mongo_cas')
    step_df = step_df.reset_index(drop=True)

    if len(step_df) < 2:
        break

    step_df["CCT"] = step_df["CCT"].astype(float)
    step_df["datetime"] = step_df["datetime"].astype(str)

    start_time = step_df['datetime'].values[1].replace("-", "/")
    start_time_obj = datetime.strptime(start_time, '%Y/%m/%d %H:%M:%S')

    now_cct = step_df['CCT'].values[1]
    print(step_times, step_df)

    if 6300 >= now_cct and now_cct >= 2500:
        if min1 > now_cct:
            min1 = now_cct
            count = 0;
            print("1")
            print(start_time, now_cct, min1, count)

        else:
            count = count + 1
            print("2", count)

            # print(start_time, now_cct, min1, count)

        if count == max_count:
            start_time_obj = start_time_obj - timedelta(minutes=count)
            start_time = start_time_obj.strftime('%Y/%m/%d %H:%M:%S')

            print("일출, 배치 최저점, 실시간 최저점")
            # print(start_time, now_cct, min1, count)
            break

    # 요것도 한번 고쳐야함.
    elif am_cut_line < start_time_obj:
        start_time_obj = start_time_obj - timedelta(minutes=count)
        start_time = start_time_obj.strftime('%Y/%m/%d %H:%M:%S')

        print("일출, 배치 최저점, 실시간 최저점111")
        # print(start_time, now_cct, min1, count)
        break

    elif count > 0:
        count = count + 1
        # print(start_time, now_cct, min1, count)

# 그걸로 부팅데이터를 만들어야함.
# 부팅 순서는 아래와 같음. 100개 확보, 역포물선 만들기, 그걸로 tadgan 돌리기, -> 반복해서 n차까지 데이터 확보하기.

# 최저점으로 부터 100개만 뽑기.
print(start_time, end_time)
boot_times = mdb.make_ktc_to_utc_4boot(start_time)
boot_data = mdb.load_last100_cct(boot_times)
boot_df = mdb.mongodb_to_df(boot_data, 'mongo_cas')
print(boot_df)
boot_df["CCT"] = boot_df["CCT"].astype(float)
# boot_df.to_csv("boot_df_df.csv", sep=',', na_rep='NaN')

# 역포물선 만들기
scaler = MinMaxScaler(feature_range=(0, 1))

temp_df = boot_df.reset_index(drop=True)
temp_df['Timestamp'] = temp_df.apply(lambda x: float(x.name) + 1, axis=1)
temp_df = temp_df[['Timestamp', 'CCT']]
temp_np = temp_df.values
temp_np = scaler.fit_transform(temp_np)
temp_df = pd.DataFrame(temp_np, columns=['Timestamp', 'input'])

reverse_df = bm.make_underzero_reverse_df(temp_df.copy(), "input")
total_df = reverse_df.append(temp_df)
total_df = total_df.reset_index(drop=True)
# 원본 시간하고 색온도 추가해서 저장하기.
for i in range(100, 200):
    total_df.loc[i, "datetime"] = boot_df['datetime'][i - 100:i - 99].values[0]
    total_df.loc[i, "ori_cct"] = boot_df['CCT'][i - 100:i - 99].values[0]
total_df = total_df[['datetime', 'ori_cct', 'Timestamp', 'input']]
total_df['Timestamp'] = total_df.apply(lambda x: float(x.name) + 1, axis=1)

# 모델 세팅
hyperparameters["epochs"] = 100
hyperparameters["shape"] = (100, 1)  # based on the window size
hyperparameters["optimizer"] = "keras.optimizers.Adam"
hyperparameters["learning_rate"] = 0.0005
hyperparameters["latent_dim"] = 20
hyperparameters["batch_size"] = 64
tgan = TadGAN(**hyperparameters)

tgan._build_tadgan()

In [ ]:
tgan.encoder.load_weights(f"{model_path}\\encoder_save_weights_{model_name}.h5")
tgan.generator.load_weights(f"{model_path}\\generator_save_weights_{model_name}.h5")
tgan.critic_x.load_weights(f"{model_path}\\critic_x_save_weights_{model_name}.h5")
tgan.critic_z.load_weights(f"{model_path}\\critic_z_save_weights_{model_name}.h5")

if model_type == "2%":
    tgan.encoder.load_weights(
        "C:\\Users\\GAKA\\Desktop\\TadGAN\\5-2. ver_2per\\model\\encoder_save_weights_5_2_2per.h5")
    tgan.generator.load_weights(
        "C:\\Users\\GAKA\\Desktop\\TadGAN\\5-2. ver_2per\\model\\generator_save_weights_5_2_2per.h5")
    tgan.critic_x.load_weights(
        "C:\\Users\\GAKA\\Desktop\\TadGAN\\5-2. ver_2per\\model\\critic_x_save_weights_5_2_2per.h5")
    tgan.critic_z.load_weights(
        "C:\\Users\\GAKA\\Desktop\\TadGAN\\5-2. ver_2per\\model\\critic_z_save_weights_5_2_2per.h5")

elif model_type == "0%":
    tgan.encoder.load_weights(
        "C:\\Users\\GAKA\\Desktop\\TadGAN\\5-2. under_reverse_oneday_sc_insert_gap\\model\\encoder_save_weights_under_reverse_oneday_sc_insert_gap_ver2.h5")
    tgan.generator.load_weights(
        "C:\\Users\\GAKA\\Desktop\\TadGAN\\5-2. under_reverse_oneday_sc_insert_gap\\model\\generator_save_weights_under_reverse_oneday_sc_insert_gap_ver2.h5")
    tgan.critic_x.load_weights(
        "C:\\Users\\GAKA\\Desktop\\TadGAN\\5-2. under_reverse_oneday_sc_insert_gap\\model\\critic_x_save_weights_under_reverse_oneday_sc_insert_gap_ver2.h5")
    tgan.critic_z.load_weights(
        "C:\\Users\\GAKA\\Desktop\\TadGAN\\5-2. under_reverse_oneday_sc_insert_gap\\model\\critic_z_save_weights_under_reverse_oneday_sc_insert_gap_ver2.h5")

elif model_type == "5%":
    tgan.encoder.load_weights(
        "C:\\Users\\GAKA\\Desktop\\TadGAN\\5-2. ver_5per\\model\\encoder_save_weights_5_2_5per.h5")
    tgan.generator.load_weights(
        "C:\\Users\\GAKA\\Desktop\\TadGAN\\5-2. ver_5per\\model\\generator_save_weights_5_2_5per.h5")
    tgan.critic_x.load_weights(
        "C:\\Users\\GAKA\\Desktop\\TadGAN\\5-2. ver_5per\\model\\critic_x_save_weights_5_2_5per.h5")
    tgan.critic_z.load_weights(
        "C:\\Users\\GAKA\\Desktop\\TadGAN\\5-2. ver_5per\\model\\critic_z_save_weights_5_2_5per.h5")

elif model_type == "10%":
    tgan.encoder.load_weights(
        "C:\\Users\\GAKA\\Desktop\\TadGAN\\5-2. ver_10per\\model\\encoder_save_weights_5_2_10per.h5")
    tgan.generator.load_weights(
        "C:\\Users\\GAKA\\Desktop\\TadGAN\\5-2. ver_10per\\model\\generator_save_weights_5_2_10per.h5")
    tgan.critic_x.load_weights(
        "C:\\Users\\GAKA\\Desktop\\TadGAN\\5-2. ver_10per\\model\\critic_x_save_weights_5_2_10per.h5")
    tgan.critic_z.load_weights(
        "C:\\Users\\GAKA\\Desktop\\TadGAN\\5-2. ver_10per\\model\\critic_z_save_weights_5_2_10per.h5")

elif model_type == "20%":
    tgan.encoder.load_weights(
        "C:\\Users\\GAKA\\Desktop\\TadGAN\\5-2. ver_20per\\model\\encoder_save_weights_5_2_20per.h5")
    tgan.generator.load_weights(
        "C:\\Users\\GAKA\\Desktop\\TadGAN\\5-2. ver_20per\\model\\generator_save_weights_5_2_20per.h5")
    tgan.critic_x.load_weights(
        "C:\\Users\\GAKA\\Desktop\\TadGAN\\5-2. ver_20per\\model\\critic_x_save_weights_5_2_20per.h5")
    tgan.critic_z.load_weights(
        "C:\\Users\\GAKA\\Desktop\\TadGAN\\5-2. ver_20per\\model\\critic_z_save_weights_5_2_20per.h5")

elif model_type == "30%":
    tgan.encoder.load_weights(
        "C:\\Users\\GAKA\\Desktop\\TadGAN\\5-2. ver_30per\\model\\encoder_save_weights_5_2_30per.h5")
    tgan.generator.load_weights(
        "C:\\Users\\GAKA\\Desktop\\TadGAN\\5-2. ver_30per\\model\\generator_save_weights_5_2_30per.h5")
    tgan.critic_x.load_weights(
        "C:\\Users\\GAKA\\Desktop\\TadGAN\\5-2. ver_30per\\model\\critic_x_save_weights_5_2_30per.h5")
    tgan.critic_z.load_weights(
        "C:\\Users\\GAKA\\Desktop\\TadGAN\\5-2. ver_30per\\model\\critic_z_save_weights_5_2_30per.h5")

elif model_type == "60%":
    tgan.encoder.load_weights(
        "C:\\Users\\GAKA\\Desktop\\TadGAN\\5-2. ver_60per\\model\\encoder_save_weights_5_2_60per.h5")
    tgan.generator.load_weights(
        "C:\\Users\\GAKA\\Desktop\\TadGAN\\5-2. ver_60per\\model\\generator_save_weights_5_2_60per.h5")
    tgan.critic_x.load_weights(
        "C:\\Users\\GAKA\\Desktop\\TadGAN\\5-2. ver_60per\\model\\critic_x_save_weights_5_2_60per.h5")
    tgan.critic_z.load_weights(
        "C:\\Users\\GAKA\\Desktop\\TadGAN\\5-2. ver_60per\\model\\critic_z_save_weights_5_2_60per.h5")

elif model_type == "15%_new":
    tgan.encoder.load_weights(
        "C:\\Users\\GAKA\\Desktop\\TadGAN\\5-2. ver_15per(new)\\model\\encoder_save_weights_5_2_15per(new).h5")
    tgan.generator.load_weights(
        "C:\\Users\\GAKA\\Desktop\\TadGAN\\5-2. ver_15per(new)\\model\\generator_save_weights_5_2_15per(new).h5")
    tgan.critic_x.load_weights(
        "C:\\Users\\GAKA\\Desktop\\TadGAN\\5-2. ver_15per(new)\\model\\critic_x_save_weights_5_2_15per(new).h5")
    tgan.critic_z.load_weights(
        "C:\\Users\\GAKA\\Desktop\\TadGAN\\5-2. ver_15per(new)\\model\\critic_z_save_weights_5_2_15per(new).h5")

elif model_type == "15%_all":
    tgan.encoder.load_weights(
        "C:\\Users\\GAKA\\Desktop\\TadGAN\\5-2. ver_15per\\model\\encoder_save_weights_5_2_15per.h5")
    tgan.generator.load_weights(
        "C:\\Users\\GAKA\\Desktop\\TadGAN\\5-2. ver_15per\\model\\generator_save_weights_5_2_15per.h5")
    tgan.critic_x.load_weights(
        "C:\\Users\\GAKA\\Desktop\\TadGAN\\5-2. ver_15per\\model\\critic_x_save_weights_5_2_15per.h5")
    tgan.critic_z.load_weights(
        "C:\\Users\\GAKA\\Desktop\\TadGAN\\5-2. ver_15per\\model\\critic_z_save_weights_5_2_15per.h5")

elif model_type == "15%_old":
    tgan.encoder.load_weights(
        "C:\\Users\\GAKA\\Desktop\\TadGAN\\5-2. ver_15per(old)\\model\\encoder_save_weights_5_2_15per(old).h5")
    tgan.generator.load_weights(
        "C:\\Users\\GAKA\\Desktop\\TadGAN\\5-2. ver_15per(old)\\model\\generator_save_weights_5_2_15per(old).h5")
    tgan.critic_x.load_weights(
        "C:\\Users\\GAKA\\Desktop\\TadGAN\\5-2. ver_15per(old)\\model\\critic_x_save_weights_5_2_15per(old).h5")
    tgan.critic_z.load_weights(
        "C:\\Users\\GAKA\\Desktop\\TadGAN\\5-2. ver_15per(old)\\model\\critic_z_save_weights_5_2_15per(old).h5")

elif model_type == "10%_new":
    tgan.encoder.load_weights(
        "C:\\Users\\GAKA\\Desktop\\TadGAN\\5-2. ver_10per(new)\\model\\encoder_save_weights_5_2_10per(new).h5")
    tgan.generator.load_weights(
        "C:\\Users\\GAKA\\Desktop\\TadGAN\\5-2. ver_10per(new)\\model\\generator_save_weights_5_2_10per(new).h5")
    tgan.critic_x.load_weights(
        "C:\\Users\\GAKA\\Desktop\\TadGAN\\5-2. ver_10per(new)\\model\\critic_x_save_weights_5_2_10per(new).h5")
    tgan.critic_z.load_weights(
        "C:\\Users\\GAKA\\Desktop\\TadGAN\\5-2. ver_10per(new)\\model\\critic_z_save_weights_5_2_10per(new).h5")

elif model_type == "10%_all2":
    tgan.encoder.load_weights(
        "C:\\Users\\GAKA\\Desktop\\TadGAN\\5-2. ver_10per(all2)\\model\\encoder_save_weights_5_2_10per(all2).h5")
    tgan.generator.load_weights(
        "C:\\Users\\GAKA\\Desktop\\TadGAN\\5-2. ver_10per(all2)\\model\\generator_save_weights_5_2_10per(all2).h5")
    tgan.critic_x.load_weights(
        "C:\\Users\\GAKA\\Desktop\\TadGAN\\5-2. ver_10per(all2)\\model\\critic_x_save_weights_5_2_10per(all2).h5")
    tgan.critic_z.load_weights(
        "C:\\Users\\GAKA\\Desktop\\TadGAN\\5-2. ver_10per(all2)\\model\\critic_z_save_weights_5_2_10per(all2).h5")

elif model_type == "10%_old":
    tgan.encoder.load_weights(
        "C:\\Users\\GAKA\\Desktop\\TadGAN\\5-2. ver_10per(old)\\model\\encoder_save_weights_5_2_10per(old).h5")
    tgan.generator.load_weights(
        "C:\\Users\\GAKA\\Desktop\\TadGAN\\5-2. ver_10per(old)\\model\\generator_save_weights_5_2_10per(old).h5")
    tgan.critic_x.load_weights(
        "C:\\Users\\GAKA\\Desktop\\TadGAN\\5-2. ver_10per(old)\\model\\critic_x_save_weights_5_2_10per(old).h5")
    tgan.critic_z.load_weights(
        "C:\\Users\\GAKA\\Desktop\\TadGAN\\5-2. ver_10per(old)\\model\\critic_z_save_weights_5_2_10per(old).h5")

elif model_type == "50%_new":
    tgan.encoder.load_weights(
        "C:\\Users\\GAKA\\Desktop\\TadGAN\\5-2. ver_50per(new)\\model\\encoder_save_weights_5_2_50per(new).h5")
    tgan.generator.load_weights(
        "C:\\Users\\GAKA\\Desktop\\TadGAN\\5-2. ver_50per(new)\\model\\generator_save_weights_5_2_50per(new).h5")
    tgan.critic_x.load_weights(
        "C:\\Users\\GAKA\\Desktop\\TadGAN\\5-2. ver_50per(new)\\model\\critic_x_save_weights_5_2_50per(new).h5")
    tgan.critic_z.load_weights(
        "C:\\Users\\GAKA\\Desktop\\TadGAN\\5-2. ver_50per(new)\\model\\critic_z_save_weights_5_2_50per(new).h5")

elif model_type == "50%_all":
    tgan.encoder.load_weights(
        "C:\\Users\\GAKA\\Desktop\\TadGAN\\5-2. ver_50per(all)\\model\\encoder_save_weights_5_2_50per(all).h5")
    tgan.generator.load_weights(
        "C:\\Users\\GAKA\\Desktop\\TadGAN\\5-2. ver_50per(all)\\model\\generator_save_weights_5_2_50per(all).h5")
    tgan.critic_x.load_weights(
        "C:\\Users\\GAKA\\Desktop\\TadGAN\\5-2. ver_50per(all)\\model\\critic_x_save_weights_5_2_50per(all).h5")
    tgan.critic_z.load_weights(
        "C:\\Users\\GAKA\\Desktop\\TadGAN\\5-2. ver_50per(all)\\model\\critic_z_save_weights_5_2_50per(all).h5")

elif model_type == "50%_old":
    tgan.encoder.load_weights(
        "C:\\Users\\GAKA\\Desktop\\TadGAN\\5-2. ver_50per(old)\\model\\encoder_save_weights_5_2_50per(old).h5")
    tgan.generator.load_weights(
        "C:\\Users\\GAKA\\Desktop\\TadGAN\\5-2. ver_50per(old)\\model\\generator_save_weights_5_2_50per(old).h5")
    tgan.critic_x.load_weights(
        "C:\\Users\\GAKA\\Desktop\\TadGAN\\5-2. ver_50per(old)\\model\\critic_x_save_weights_5_2_50per(old).h5")
    tgan.critic_z.load_weights(
        "C:\\Users\\GAKA\\Desktop\\TadGAN\\5-2. ver_50per(old)\\model\\critic_z_save_weights_5_2_50per(old).h5")

elif model_type == "0%_new":
    tgan.encoder.load_weights(
        "C:\\Users\\GAKA\\Desktop\\TadGAN\\5-2. ver_0per(new)\\model\\encoder_save_weights_5_2_0per(new).h5")
    tgan.generator.load_weights(
        "C:\\Users\\GAKA\\Desktop\\TadGAN\\5-2. ver_0per(new)\\model\\generator_save_weights_5_2_0per(new).h5")
    tgan.critic_x.load_weights(
        "C:\\Users\\GAKA\\Desktop\\TadGAN\\5-2. ver_0per(new)\\model\\critic_x_save_weights_5_2_0per(new).h5")
    tgan.critic_z.load_weights(
        "C:\\Users\\GAKA\\Desktop\\TadGAN\\5-2. ver_0per(new)\\model\\critic_z_save_weights_5_2_0per(new).h5")

elif model_type == "2%_new":
    tgan.encoder.load_weights(
        "C:\\Users\\GAKA\\Desktop\\TadGAN\\5-2. ver_2per(new)\\model\\encoder_save_weights_5_2_2per(new).h5")
    tgan.generator.load_weights(
        "C:\\Users\\GAKA\\Desktop\\TadGAN\\5-2. ver_2per(new)\\model\\generator_save_weights_5_2_2per(new).h5")
    tgan.critic_x.load_weights(
        "C:\\Users\\GAKA\\Desktop\\TadGAN\\5-2. ver_2per(new)\\model\\critic_x_save_weights_5_2_2per(new).h5")
    tgan.critic_z.load_weights(
        "C:\\Users\\GAKA\\Desktop\\TadGAN\\5-2. ver_2per(new)\\model\\critic_z_save_weights_5_2_2per(new).h5")

elif model_type == "5%_new":
    tgan.encoder.load_weights(
        "C:\\Users\\GAKA\\Desktop\\TadGAN\\5-2. ver_5per(new)\\model\\encoder_save_weights_5_2_5per(new).h5")
    tgan.generator.load_weights(
        "C:\\Users\\GAKA\\Desktop\\TadGAN\\5-2. ver_5per(new)\\model\\generator_save_weights_5_2_5per(new).h5")
    tgan.critic_x.load_weights(
        "C:\\Users\\GAKA\\Desktop\\TadGAN\\5-2. ver_5per(new)\\model\\critic_x_save_weights_5_2_5per(new).h5")
    tgan.critic_z.load_weights(
        "C:\\Users\\GAKA\\Desktop\\TadGAN\\5-2. ver_5per(new)\\model\\critic_z_save_weights_5_2_5per(new).h5")

elif model_type == "20%_new":
    tgan.encoder.load_weights(
        "C:\\Users\\GAKA\\Desktop\\TadGAN\\5-2. ver_20per(new)\\model\\encoder_save_weights_5_2_20per(new).h5")
    tgan.generator.load_weights(
        "C:\\Users\\GAKA\\Desktop\\TadGAN\\5-2. ver_20per(new)\\model\\generator_save_weights_5_2_20per(new).h5")
    tgan.critic_x.load_weights(
        "C:\\Users\\GAKA\\Desktop\\TadGAN\\5-2. ver_20per(new)\\model\\critic_x_save_weights_5_2_20per(new).h5")
    tgan.critic_z.load_weights(
        "C:\\Users\\GAKA\\Desktop\\TadGAN\\5-2. ver_20per(new)\\model\\critic_z_save_weights_5_2_20per(new).h5")

elif model_type == "30%_new":
    tgan.encoder.load_weights(
        "C:\\Users\\GAKA\\Desktop\\TadGAN\\5-2. ver_30per(new)\\model\\encoder_save_weights_5_2_30per(new).h5")
    tgan.generator.load_weights(
        "C:\\Users\\GAKA\\Desktop\\TadGAN\\5-2. ver_30per(new)\\model\\generator_save_weights_5_2_30per(new).h5")
    tgan.critic_x.load_weights(
        "C:\\Users\\GAKA\\Desktop\\TadGAN\\5-2. ver_30per(new)\\model\\critic_x_save_weights_5_2_30per(new).h5")
    tgan.critic_z.load_weights(
        "C:\\Users\\GAKA\\Desktop\\TadGAN\\5-2. ver_30per(new)\\model\\critic_z_save_weights_5_2_30per(new).h5")

In [ ]:

#  기존엔 스케일을 한번에 0,1 로 줄여서 반복후에 스케일업, 역스케일링을 했다.
#  실시간으로 하게되면 1회 실행했을 때 이후 들어오는 데이터가 더 높은 값일 수도 있기 때문에 다시 스케일을 맞춰야 한다.
for i in range(RE_MAX_TIME):
    X, index = time_segments_aggregate(total_df[['Timestamp', 'input']], interval=1, time_column='Timestamp')
    imp = SimpleImputer()
    X = imp.fit_transform(X)

    X, y, X_index, y_index = rolling_window_sequences(X, index,
                                                      window_size=100,
                                                      target_size=1,
                                                      step_size=1,
                                                      target_column=0)

    # 학습된 모델을 데이터에 적용
    X_hat, critic = tgan.predict(X)
    y_hat = unroll_ts(X_hat)
    # plot_ts([y, y_hat], labels=['original', 'reconstructed'])

    total_df.loc[100:200, "tadgan_temp"] = y_hat
    # print(y_hat)
    #

    # 역스케일링 하고, re_n차에 저장, 그걸 가지고 다시 0,1 스케일링, 역포물선 제작, input으로 넣기
    if i == 0:
        max_cct = np.nanmax(total_df['ori_cct'].values)
        min_cct = np.nanmin(total_df['ori_cct'].values)
    else:
        max_cct = np.nanmax(total_df['re_' + str(i)].values)
        min_cct = np.nanmin(total_df['re_' + str(i)].values)

    total_df['re_' + str(i)] = total_df.apply(lambda x: (x['input'] * (max_cct - min_cct)) + min_cct, axis=1)
    total_df['re_' + str(i + 1)] = total_df.apply(lambda x: (x['tadgan_temp'] * (max_cct - min_cct)) + min_cct,
                                                  axis=1)

    reverse_df = bm.make_Relatively_reverse_df(total_df.copy(), 're_' + str(i + 1))
    reverse_df = reverse_df.reset_index(drop=True)

    temp_max = np.nanmax(reverse_df['re_' + str(i + 1)].values)
    temp_min = np.nanmin(total_df['re_' + str(i + 1)].values)
    reverse_df['re_' + str(i + 1)] = reverse_df.apply(lambda x: x['re_' + str(i + 1)] - (temp_max - temp_min),
                                                      axis=1)

    total_df.loc[0:99, 're_' + str(i + 1)] = reverse_df['re_' + str(i + 1)][0:]

    temp_df = total_df[['re_' + str(i + 1)]]
    temp_np = temp_df.values
    temp_np = scaler.fit_transform(temp_np)
    temp_df = pd.DataFrame(temp_np, columns=['input'])
    total_df.loc[0:, "input"] = temp_df["input"][0:]

In [ ]:

#  부팅 데이터 적용하고, 그거 기준으로 스케일 업 하는 횟수 정하고, 역스케일링 하고 반복.

# 스케일 업하는 기준 잡기(지수)
total_df['up_sc'] = total_df.apply(lambda x: x[before_N] / x[target_N], axis=1)

#
sc_k_pd = pd.DataFrame()
sc_k_pd['re_0'] = total_df['re_0']

row1 = []
row2 = []
for j in range(41):
    # 각 업스케일링 회차에 대한 성능평가 진행하여 직전 대체값에서 얼마나 변화가 적은지 판단하고 삽입. 반복.

    SC_K = 0.1 * j
    sc_k_pd[str(SC_K)] = total_df.apply(lambda x: (float(x[target_N]) * pow(float(x['up_sc']), SC_K)), axis=1)
    # print(sc_k_pd[str(SC_K)])
    sc_k_pd[str(SC_K) + "_AE"] = total_df.apply(lambda x: 0, axis=1)

    for i in range(1, len(sc_k_pd)):
        sc_k_pd.loc[i, str(SC_K) + '_AE'] = abs((float(sc_k_pd['re_0'][i]) - (float(sc_k_pd[str(SC_K)][i]))))
    #
    row1.append(SC_K)
    row2.append(np.mean(sc_k_pd[str(SC_K) + '_AE'][99:].to_numpy()))

    # plt.scatter(range(len(sc_k_pd[99:])), sc_k_pd['re_0'][99:], color='red', marker='o', s=1)
    # plt.scatter(range(len(sc_k_pd[99:])), sc_k_pd[str(SC_K)][99:], color='blue', marker='o', s=1)
    # plt.title(str(SC_K))
    # plt.savefig("C:\\Users\\GAKA\\Desktop\\workspace\\Python\\TadGAN_final\\\RealTime_system\\plt\\"+str(SC_K)+".jpg")
    # plt.close("all")

index = row2.index(min(row2))
SC_K = row1[index]
print(SC_K)
total_df['output'] = total_df.apply(lambda x: x[target_N] * pow(x['up_sc'], SC_K), axis=1)
total_df = total_df.drop(columns=['input', 'tadgan_temp'])
total_df.to_csv("booting.csv", sep=',')

# return 0
#
#
#
#
#
#

start_time = total_df['datetime'].values[-1]
time = str(start_time)
temp = time.split("T")
date = temp[0].split("-")

ori_date = date[0] + "/" + date[1] + "/" + date[2]
start_time = ori_date + " " + temp[1].split(".")[0]
start_time_obj = datetime.strptime(start_time, '%Y/%m/%d %H:%M:%S')

# 이제 데이터 1개씩 끌어와서 반복 실행.
#
j = 100
min1 = 1000000
count = 0
find_endpoint = False
end_point = 0
# total_df['J_index'] = temp_df.apply(lambda x: 0, axis=1)

In [ ]:
while True:
    # start_time_obj를 증가해시켜야함.(맨뒤에서)
    if (find_endpoint==False):
        if 1 > ((end_time_obj - start_time_obj).seconds / 60):
            print("그냥 다돔")
            break

        elif end_time_obj < start_time_obj:
            print("그냥 다돔1", j, start_time_obj)
            break

        step_times = mdb.make_ktc_to_utc_4step1(start_time)
        step_data = mdb.load_last1_cct1(step_times)

        step_df = mdb.mongodb_to_df(step_data, 'mongo_cas')
        step_df = step_df.reset_index(drop=True)

        if len(step_df) < 2:
            break

        step_df["CCT"] = step_df["CCT"].astype(float)
        step_df["datetime"] = step_df["datetime"].astype(str)

        # 요건 밑으로 이전해야 할 수도 있고 알고리즘 내부에서 실행 해야 할 수도 있음.
        start_time = step_df['datetime'].values[1].replace("-", "/")
        start_time_obj = datetime.strptime(start_time, '%Y/%m/%d %H:%M:%S')

        CCT = step_df['CCT'].values[1]

        if count < max_count:
            total_df.loc[j + 100, "ori_cct"] = CCT
            total_df.loc[j + 100, "re_0"] = CCT
            total_df.loc[j + 100, "datetime"] = start_time
            total_df['Timestamp'] = temp_df.apply(lambda x: float(x.name) + 1, axis=1)

    # print(start_time_obj)
    if (pm_cut_line <= start_time_obj):
        # 의심구간 처음 들어옴

        if 1 > abs((pm_cut_line - start_time_obj).seconds / 60):
            if 6300 < CCT and CCT < 2500:
                min1 = CCT
                count = count + 1
        elif count > 0 and 6300 < CCT and CCT < 2500:
            count = count + 1
            if count == max_count:
                end_time_obj = start_time_obj - timedelta(minutes=count)
                end_time = end_time_obj.strftime('%Y/%m/%d %H:%M:%S')

                print("일몰, 배치 최저점, 실시간 최저점1")
                find_endpoint = True
                end_point = j + 100 - k
                print(end_time)
                count = count + 1
                for k in range(max_count):
                    total_df = total_df.drop([j + 100 - k])
                temp = bm.make_Relatively_reverse_df_4ending(total_df)

                input_count = 0
                for i in range(len(total_df), len(total_df) + 100):
                    total_df.loc[i, "ori_cct"] = temp['CCT'][input_count]
                    total_df.loc[i, "re_0"] = temp['CCT'][input_count]
                    input_count = input_count + 1
                total_df['Timestamp'] = temp_df.apply(lambda x: float(x.name) + 1, axis=1)
                # break

        if 6300 >= CCT and CCT >= 2500 and count < max_count:
            if min1 > CCT:
                min1 = CCT
                count = 0

            else:
                count = count + 1

            if count == max_count:
                end_time_obj = start_time_obj - timedelta(minutes=count)
                end_time = end_time_obj.strftime('%Y/%m/%d %H:%M:%S')

                print("일몰, 배치 최저점, 실시간 최저점2")
                find_endpoint = True
                end_point = j + 100 - k
                print(end_time)
                # print(total_df)

                count = count + 1
                for k in range(max_count):
                    total_df = total_df.drop([j + 100 - k])
                temp = bm.make_Relatively_reverse_df_4ending(total_df)

                input_count = 0
                for i in range(len(total_df), len(total_df) + 100):
                    total_df.loc[i, "ori_cct"] = temp['CCT'][input_count]
                    total_df.loc[i, "re_0"] = temp['CCT'][input_count]
                    input_count = input_count + 1
                total_df['Timestamp'] = temp_df.apply(lambda x: float(x.name) + 1, axis=1)
                # break

    # 임시 점프구간
    # j = j+1
    # continue
    #

    # 모델 반복 적용
    # total_df['J_index'][j] = j
    indexing = 100
    if find_endpoint:
        if pd.isnull(total_df.loc[j, 'datetime']):
            print(total_df.loc[j,'datetime'])
            print(j, end_point)
            break

    # print(j, len(temp_df))
    for i in range(RE_MAX_TIME):
        temp_df = total_df.loc[j - indexing: j + indexing, ['Timestamp', 're_' + str(i)]]
        # print(j, len(temp_df))
        # if len(temp_df) != 201:
        #     print(j, len(temp_df))
            # 백개보다 아래로 내려가면 멈추긴해야지.. 왜냐하면 앤딩데이터가 만들어주니깐..~! 이건 유지.
            # break

        max_cct = np.nanmax(temp_df['re_' + str(i)].values)
        min_cct = np.nanmin(temp_df['re_' + str(i)].values)

        temp_np = temp_df.values
        temp_np = scaler.fit_transform(temp_np)
        temp_df = pd.DataFrame(temp_np, columns=['Timestamp', 'input'])
        temp_df['Timestamp'] = temp_df.apply(lambda x: float(x.name) + 1, axis=1)
        # print(len(temp_df))
        X, index = time_segments_aggregate(temp_df, interval=1, time_column='Timestamp')
        imp = SimpleImputer()
        X = imp.fit_transform(X)
        # print(X)
        X, y, X_index, y_index = rolling_window_sequences(X, index,
                                                          window_size=100,
                                                          target_size=1,
                                                          step_size=1,
                                                          target_column=0)

        # 학습된 모델을 데이터에 적용
        X_hat, critic = tgan.predict(X)
        y_hat = unroll_ts(X_hat)
        # print(len(y_hat))
        # plot_ts([y, y_hat], labels=['original', 'reconstructed'])

        # 스케일 업해서. 저장해야함.

        # y hat이 j - indexing-100 개만큼 있음. total[j - indexing+100:] 범위겠지.
        for k in range(len(y_hat)):
            temp_cct = (y_hat[k] * (max_cct - min_cct)) + min_cct
            total_df.loc[j + k, 're_' + str(i + 1)] = temp_cct

        # temp_cct = (y_hat[0] * (max_cct - min_cct)) + min_cct
        # total_df.loc[j, 're_' + str(i + 1)] = temp_cct
        # print(max_cct,min_cct,y_hat,temp_cct)

    if len(temp_df) > 100:
        # 산출물이 100개 이상일 때에만 결과물을 넣는다..? 근데 이게 꼭 필요한지는 의문 .,., 2중 보호같이 되어있어서
        total_df.loc[j, 'up_sc'] = total_df.loc[j, before_N] / total_df.loc[j, target_N]
        total_df.loc[j, 'output'] = total_df.loc[j, target_N] * pow(total_df.loc[j, 'up_sc'], SC_K)

    print(j)
    # if j%20==0:
    #     plt.scatter(range(j), total_df['log_5'][:j], color='red', marker='o', s=1)
    #     plt.scatter(range(j), total_df['ori_cct'][:j], color='blue', marker='o', s=1)
    #     plt.title('log_5_'+str(j))
    #     plt.savefig("C:\\Users\\GAKA\\Desktop\\workspace\\Python\\TadGAN_final\\\RealTime_system\\plt\\" + 'log_5_'+str(j) + ".jpg")
    #     plt.close("all")
    j = j + 1

# print(total_df)
# 이건 마지막 끝나고 잔여 데이터 100개 넣어주는 행위 / 삭제 예정 : 앤딩 데이터셋이 있으니깐.!


# for q in range(j, len(total_df)):
#     total_df.loc[q, 'up_sc'] = total_df.loc[q, before_N] / total_df.loc[q, target_N]
#     total_df.loc[q, 'output'] = total_df.loc[q, target_N] * pow(total_df.loc[q, 'up_sc'], SC_K)
total_df = total_df[total_df['datetime'].notnull()]

total_df.to_csv("./result/" + ori_date.replace("/", "-") + "_" + model_type + ".csv", sep=',')